In [1]:
import pandas as pd
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import accuracy_score

def convert_label(value_map):
    def mapper(label):
        if label in value_map:
            return value_map[label]
        else:
            return label
    return mapper


def auto_discretize_column(df, column_name, num_bins=3):
    # Get the column data
    col_data = df[column_name]
    
    # Determine the bin ranges based on the data distribution
    min_val = col_data.min()
    max_val = col_data.max()
    
    
    bin_size = (max_val - min_val) / num_bins
    bins = [min_val + i * bin_size for i in range(num_bins + 1)]
    bins[0] -= max_val
    bins[num_bins] += max_val
    
    return bins


def fill_missing_values(df, columns_to_fill):
    for col in columns_to_fill:
        # Get the columns that are related to the current column
        related_cols = [c for c in df.columns if c != col and c in df.columns]
        
        # Calculate the mean of the related columns
        mean_value = df[related_cols].mean(axis=1)
        
        # Fill the missing values with the mean
        df[col] = df[col].fillna(mean_value)
    
    return df

In [2]:
# Load the data into a pandas DataFrame
df = pd.read_excel('az12.xlsx')
data = df.copy()
data.head()

,Well ID,X,Y,DEPTH,CALI,DT,NPHI,PHIE,RHOB,SW,litho,VANH,VCLC,VDOL,VOL_QUARTZ,VOL_SHALE,VSAN .V/V
0,AZ-012,601341.284391,2.055966e+06,2571.1404,-999.25,-999.25,0.0865,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,0,-999.25,-999.25
1,AZ-012,601341.284391,2.055966e+06,2571.2928,-999.25,-999.25,0.0872,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,0,-999.25,-999.25
2,AZ-012,601341.284391,2.055966e+06,2571.4452,-999.25,-999.25,0.0876,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,0,-999.25,-999.25
3,AZ-012,601341.284391,2.055966e+06,2571.5976,-999.25,-999.25,0.0888,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,0,-999.25,-999.25
4,AZ-012,601341.284391,2.055966e+06,2571.7500,-999.25,-999.25,0.0903,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,0,-999.25,-999.25


# Drop data

In [3]:
data = data.drop('Well ID', axis=1)
data = data.drop('VOL_QUARTZ', axis=1)
data = data.drop('X', axis=1)
data = data.drop('Y', axis=1)
data = data.drop('VANH', axis=1)


In [4]:
data.head()

,DEPTH,CALI,DT,NPHI,PHIE,RHOB,SW,litho,VCLC,VDOL,VOL_SHALE,VSAN .V/V
0,2571.1404,-999.25,-999.25,0.0865,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
1,2571.2928,-999.25,-999.25,0.0872,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
2,2571.4452,-999.25,-999.25,0.0876,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
3,2571.5976,-999.25,-999.25,0.0888,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
4,2571.7500,-999.25,-999.25,0.0903,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25


In [5]:
data = data[data['litho'] != -999.25]
data.head()

,DEPTH,CALI,DT,NPHI,PHIE,RHOB,SW,litho,VCLC,VDOL,VOL_SHALE,VSAN .V/V
63,2580.7416,5.7001,52.4181,0.0803,0.0622,2.9844,1.000,ANHYDRIDE,0.0,0.0334,0.0,-999.25
64,2580.8940,5.7442,51.0048,0.0365,0.0388,2.9814,1.000,ANHYDRIDE,0.0,0.0013,0.0,-999.25
65,2581.0464,5.6378,52.3359,0.0164,0.0281,2.9770,1.000,ANHYDRIDE,0.0,0.0000,0.0,-999.25
66,2581.1988,5.5114,53.6957,0.0149,0.0281,2.9693,1.000,ANHYDRIDE,0.0,0.0000,0.0,-999.25
67,2581.3512,5.6852,54.0347,0.0133,0.0274,2.9616,0.961,ANHYDRIDE,0.0,0.0000,0.0,-999.25


In [6]:
len(data)

1202

In [7]:
data['VSAN .V/V'].value_counts()


VSAN .V/V
-999.2500    275
 0.0000       85
 0.0008        4
 0.0027        4
 0.7270        3
            ... 
 0.5640        1
 0.5170        1
 0.4913        1
 0.4518        1
 0.6731        1
Name: count, Length: 781, dtype: int64

In [8]:
mean_value = data[data['VCLC'] != -999.25]['VCLC'].median()
data['VCLC'] = data['VCLC'].replace(-999.25, mean_value)

mean_value = data[data['VDOL'] != -999.25]['VDOL'].median()
data['VDOL'] = data['VDOL'].replace(-999.25, mean_value)

mean_value = data[data['VSAN .V/V'] != -999.25]['VSAN .V/V'].median()
data['VSAN .V/V'] = data['VSAN .V/V'].replace(-999.25, mean_value)

data['VCLC'].value_counts()
data['VDOL'].value_counts()
data['VSAN .V/V'].value_counts()



VSAN .V/V
0.4250    276
0.0000     85
0.0008      4
0.0027      4
0.7270      3
         ... 
0.5742      1
0.5640      1
0.5170      1
0.4913      1
0.6731      1
Name: count, Length: 780, dtype: int64

# discretization 

In [9]:
unique_values = data['litho'].unique()
unique_values

array(['ANHYDRIDE', 'DOLOMITE', 'MIX', 'LIMESTONE', 'SHALE', 'SANDSTONE'],
      dtype=object)

In [10]:

data2 = data.copy()


# data2['DT2'] = data2['DT'] ** 2
# data2 = data2.drop('DT', axis=1)
# data2['DT2'] = pd.cut(data2['DT2'], bins=auto_discretize_column(data2,'DT2',7), labels=[0, 1, 2,3,4,5,6])

data2["litho"] = data2["litho"].apply(convert_label({'ANHYDRIDE': 0, 'DOLOMITE': 1,'MIX':2,'LIMESTONE':4, 'SHALE':5, 'SANDSTONE':6}))

data2['PHIE'] = pd.cut(data2['PHIE'], bins=auto_discretize_column(data2,'PHIE',7), labels=[0, 1, 2,3,4,5,6])
data2['CALI'] = pd.cut(data2['CALI'], bins=auto_discretize_column(data2,'CALI',7), labels=[0, 1, 2,3,4,5,6])

data2['DT'] = pd.cut(data2['DT'], bins=auto_discretize_column(data2,'DT',7), labels=[0, 1, 2,3,4,5,6])
data2['VOL_SHALE'] = pd.cut(data2['VOL_SHALE'], bins=auto_discretize_column(data2,'VOL_SHALE',7), labels=[0, 1, 2,3,4,5,6]) 

data2['RHOB'] = pd.cut(data2['RHOB'], bins=auto_discretize_column(data2,'RHOB',7), labels=[0, 1, 2,3,4,5,6])
data2['VDOL'] = pd.cut(data2['VDOL'], bins=auto_discretize_column(data2,'VDOL',7), labels=[0, 1, 2,3,4,5,6])




data2['DEPTH'] = pd.cut(data2['DEPTH'], bins=auto_discretize_column(data2,'DEPTH',7), labels=[0, 1, 2,3,4,5,6])
data2['NPHI'] = pd.cut(data2['NPHI'], bins=auto_discretize_column(data2,'NPHI',7), labels=[0, 1, 2,3,4,5,6])
data2['SW'] = pd.cut(data2['SW'], bins=auto_discretize_column(data2,'SW',7), labels=[0, 1, 2,3,4,5,6]) 
data2['VCLC'] = pd.cut(data2['VCLC'], bins=auto_discretize_column(data2,'VCLC',7), labels=[0, 1, 2,3,4,5,6])
data2['VSAN .V/V'] = pd.cut(data2['VSAN .V/V'], bins=auto_discretize_column(data2,'VSAN .V/V',7), labels=[0, 1, 2,3,4,5,6]) 





In [11]:
data2.tail()

,DEPTH,CALI,DT,NPHI,PHIE,RHOB,SW,litho,VCLC,VDOL,VOL_SHALE,VSAN .V/V
1260,6,0,4,1,4,0,1,6,0,0,1,6
1261,6,0,4,1,4,0,1,6,0,0,1,6
1262,6,0,5,1,5,0,2,6,0,0,0,6
1263,6,0,5,1,5,0,1,6,0,0,0,6
1264,6,0,5,1,5,0,1,6,0,0,0,6


In [12]:
data2 = data2.astype({
    'CALI': 'int64',
    # 'DT2': 'int64',
    # 'DT4': 'int64',
    'DT': 'int64',
    'NPHI': 'int64',
    'PHIE': 'int64',
    'RHOB': 'int64',
    'VDOL': 'int64',
    'SW': 'int64',
    'VCLC': 'int64',
    'VOL_SHALE': 'int64',
    'VSAN .V/V': 'int64',
    'DEPTH': 'int64'
})
data2.info()



<class 'pandas.core.frame.DataFrame'>
Index: 1202 entries, 63 to 1264
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   DEPTH      1202 non-null   int64
 1   CALI       1202 non-null   int64
 2   DT         1202 non-null   int64
 3   NPHI       1202 non-null   int64
 4   PHIE       1202 non-null   int64
 5   RHOB       1202 non-null   int64
 6   SW         1202 non-null   int64
 7   litho      1202 non-null   int64
 8   VCLC       1202 non-null   int64
 9   VDOL       1202 non-null   int64
 10  VOL_SHALE  1202 non-null   int64
 11  VSAN .V/V  1202 non-null   int64
dtypes: int64(12)
memory usage: 122.1 KB


In [13]:
data2['DT - RHOB'] = data2['DT'] - data2['RHOB']
data2 = data2.drop('DT', axis=1)
# data2 = data2.drop('RHOB', axis=1)

In [14]:
data2['DEPTH ** RHOB'] = data2['DEPTH'] ** data2['RHOB']
data2 = data2.drop('DEPTH', axis=1)
data2 = data2.drop('RHOB', axis=1)

In [15]:
# Get the number of rows and columns
num_rows, num_cols = data.shape
print(f"The DataFrame has {num_rows} rows and {num_cols} columns.")

The DataFrame has 1202 rows and 12 columns.


In [16]:
data2.info()
data = data2.copy()
data.to_csv('data_cleaned3.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 1202 entries, 63 to 1264
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   CALI           1202 non-null   int64
 1   NPHI           1202 non-null   int64
 2   PHIE           1202 non-null   int64
 3   SW             1202 non-null   int64
 4   litho          1202 non-null   int64
 5   VCLC           1202 non-null   int64
 6   VDOL           1202 non-null   int64
 7   VOL_SHALE      1202 non-null   int64
 8   VSAN .V/V      1202 non-null   int64
 9   DT - RHOB      1202 non-null   int64
 10  DEPTH ** RHOB  1202 non-null   int64
dtypes: int64(11)
memory usage: 112.7 KB
